<center><h1> Dataset Selection</h1></center>

***

Here we will see, which dataset is more biased, or gives more mean squared error with same model. I need to check with which dataset my models perform better. To check I will use a technique called cross-validation and I will use Random Forest Regressor.

In [1]:
import pandas as pd 
import pickle 
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import set_jupyter_path

from src.car_price_prediction.utils import dataset_manager, df_utils
from src.car_price_prediction.utils import df_utils

In [2]:
data = dataset_manager.get_cleaned_outliers_dataset()

def get_dropped_dataset(data):
    data_for_drop = data.copy()
    data_for_drop = data_for_drop.drop(columns=['Url', 'Photo_Urls', 'Expiration', 'Power'])
    data_for_drop = data_for_drop.dropna()
    data_for_drop.Publication = pd.to_datetime(data_for_drop.Publication)
    data_for_drop.Publication = data_for_drop.Publication.apply(lambda x: x and x.toordinal())
    data_for_drop = shuffle(data_for_drop)
    return data_for_drop

def get_max_unbiased_dataset():
    return shuffle(dataset_manager.get_processed_dataset())

data_dropped = get_dropped_dataset(data)
data_max_unbiased = get_max_unbiased_dataset()

First I will divide the data into dataset and target variables.

In [3]:
def get_X_y(data):
    X,y = df_utils.get_data_and_target(data)
    X = pd.get_dummies(X)
    return X,y

X_drop, y_drop = get_X_y(data_dropped)
X_unbias,y_unbias = get_X_y(data_max_unbiased)

Cv argument means how many folds I want to use in cross-validation, more folds mean more unbiased error, for forest I use 5, because it's computationally more expensive.

In [4]:
def print_cv_score(estimator, est_name, cv,scoring = "neg_mean_absolute_error"):
    score_drop = cross_val_score(estimator=estimator,X = X_drop,y = y_drop,cv = cv,scoring = scoring) 
    score_unbias = cross_val_score(estimator=estimator, X = X_unbias, y = y_unbias, cv = cv, scoring = scoring)
    print("MAE for dropped data for %s: \t%.3f" % (est_name,score_drop.mean()))
    print("MAE for unbiased data for %s: \t%.3f" % (est_name,score_unbias.mean()))

In [5]:
forest = RandomForestRegressor(n_estimators=30)
print_cv_score(forest,est_name="Random Forest",cv = 5)

MAE for dropped data for Random Forest: 	-2157.338
MAE for unbiased data for Random Forest: 	-1820.664


We can see that the max_unbiased_dataset is more accurate than the dropped.

In the next notebook, I will choose a best model for this dataset to use.